<a href="https://colab.research.google.com/github/RobotMa/PyTorch/blob/master/pytorch_workflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


## Linear Regression Formula

In [ ]:
# Create *known* parameters
weight = 0.7
bias = 0.3

# Create data
start = 0
end = 1
step = 0.02
X = torch.arange(start, end, step).unsqueeze(dim=1)
y = weight * X + bias

X[:10], y[:10]

(tensor([[0.0000],
         [0.0200],
         [0.0400],
         [0.0600],
         [0.0800],
         [0.1000],
         [0.1200],
         [0.1400],
         [0.1600],
         [0.1800]]),
 tensor([[0.3000],
         [0.3140],
         [0.3280],
         [0.3420],
         [0.3560],
         [0.3700],
         [0.3840],
         [0.3980],
         [0.4120],
         [0.4260]]))

In [ ]:
train_split = int(0.8 * len(X))
X_train, y_train = X[:train_split], y[:train_split]
X_test, y_test = X[train_split:], y[train_split:]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([40, 1]),
 torch.Size([40, 1]),
 torch.Size([10, 1]),
 torch.Size([10, 1]))

In [ ]:
# Create linear regression model class
class LinearRegressionModel(nn.Module):

  def __init__(self):
    super().__init__()
    # self.weights = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
    # self.bias = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
    self.linear_layer = nn.Linear(in_features=1, out_features=1)

  def forward(self, X: torch.Tensor) -> torch.Tensor:
    return self.linear_layer(X)

In [ ]:
### PyTorch model building essentials
torch.manual_seed( 42 )

model_0 = LinearRegressionModel()

model_0.state_dict()

OrderedDict([('linear_layer.weight', tensor([[0.7645]])),
             ('linear_layer.bias', tensor([0.8300]))])

In [ ]:
model_0.state_dict()

next(model_0.parameters()).device

device(type='cpu')

### Making prediction using `torch.inference_mode()`

In [ ]:
# Set up a loss
loss_fn = nn.L1Loss()

# Set up an optimizer
optimizer = torch.optim.SGD(params=model_0.parameters(), lr=0.01)

In [ ]:
torch.manual_seed(42)

### Building a training loop (and a testing loop) in PyTorch
epochs = 300

# Track different values
epoch_count = []
loss_values = []
test_loss_values = []

for epoch in range(epochs):
  model_0.train()

  y_pred = model_0(X_train)

  loss = loss_fn(y_pred, y_train)

  optimizer.zero_grad()

  loss.backward()

  optimizer.step()

  model_0.eval()

  with torch.inference_mode():
    test_pred = model_0(X_test)
    test_loss = loss_fn(test_pred, y_test)

  if epoch % 10 == 0:
    epoch_count.append(epoch)
    loss_values.append(loss)
    test_loss_values.append(test_loss)

    print(loss)
    print(model_0.state_dict())

tensor(0.5552, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.7606]])), ('linear_layer.bias', tensor([0.8200]))])
tensor(0.4400, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.7216]])), ('linear_layer.bias', tensor([0.7200]))])
tensor(0.3248, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.6826]])), ('linear_layer.bias', tensor([0.6200]))])
tensor(0.2095, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.6436]])), ('linear_layer.bias', tensor([0.5200]))])
tensor(0.0943, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.6046]])), ('linear_layer.bias', tensor([0.4200]))])
tensor(0.0239, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.5872]])), ('linear_layer.bias', tensor([0.3525]))])
tensor(0.0200, grad_fn=<MeanBackward0>)
OrderedDict([('linear_layer.weight', tensor([[0.6025]])), ('linear_layer.bias', tensor([0.3410]))])
tensor(0.0165, grad_

In [ ]:
plot_predictions(predictions=y_preds)

NameError: name 'plot_predictions' is not defined

In [ ]:
import numpy as np
plt.plot(epoch_count, np.array(torch.tensor(loss_values)), label="Train loss")
plt.plot(epoch_count, test_loss_values, label="Test loss")
plt.legend()

### Save a Model

In [ ]:
from pathlib import Path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "01_pytorch_workflow_model_0.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

print(f"Saving model to {MODEL_SAVE_PATH}")
torch.save(obj=model_0.state_dict(), f=MODEL_SAVE_PATH)

In [ ]:
loaded_model_0 = LinearRegressionModel()

print(loaded_model_0.state_dict())

loaded_model_0.load_state_dict(torch.load(f=MODEL_SAVE_PATH))

print(loaded_model_0.state_dict())

### Putting it all together